<a href="https://colab.research.google.com/github/Moifek/Gaza_Martyrs/blob/main/Gaza_Data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Extracting and cleaning data for further use and studies provided by the Pelastinien Health Ministry
Introduction:**

In this Kaggle notebook, we import, read and convert aswell as fix and clean the data.
We also showcase a few graphs but this notebook is mainly for data preparation.pip install numpy


---




In [ ]:
pip install numpy

In [ ]:
pip install pdfplumber

In [ ]:
pip install PyPDF2

In [ ]:
pip install pandas

In [6]:
# To read the PDF
import PyPDF2
import pdfplumber
import os
import pandas as pd
import numpy

Using **PdfPlubmer** we can extract the information from the tables inside the pdf to an actual list on python

In [7]:
# Extracting tables from the page

def extract_table(pdf_path, page_num):
    # Open the pdf file
    pdf = pdfplumber.open(pdf_path)
    table = []
    pages = range(page_num)
    for i in pages:
      table_page = pdf.pages[i]
      table.append(table_page.extract_tables())
    return table

#print(extract_table('/content/drive/MyDrive/gaza_clean.pdf',145))

In [54]:
table_ = extract_table('/content/drive/MyDrive/gaza_clean.pdf',145)

This step is made because the result from our function is like so [[[]]]
so we have to scope out of the redundant lists

In [64]:
data = []
for item in table_:
  data.append(item)

In [ ]:
for x in range(2):
  data2 = [item for sublist in data for item in sublist]
  data = data2
data2

Seperating the Data columns

In [446]:
Age_ = []
Name_ = []
Identifier_ = []
Gender_ = []
Serial_ = []

for x in range(1,len(data2)):
  Serial_.append(data2[x][0])
  Identifier_.append(data2[x][1])
  Age_.append(data2[x][4])
  Gender_.append(data2[x][3])
  Name_.append(data2[x][2])

Function to Reverse the arabic words and the letters aswell, PdfPlumber reads from left to right however arabic text is read from right to left

In [94]:
def ReverseArabicText(Arr):
  res = []
  for i in range(0,len(Arr)):
    text = Arr[i]
    reversed_text = ' '.join([word[::-1] for word in text.split()])
    words = reversed_text.split()
    reversed_text = ' '.join(reversed(words))
    res.append(reversed_text)

  return res

In [447]:
Name_ = ReverseArabicText(Name_)
Gender_ = ReverseArabicText(Gender_)


**Data cleaning**

We first start with replacing the text values to integer values for the age,
'ماع نم لقأ' or 'أقل من عام' means less then a year and so we replaced it to -1

In [ ]:
for x in range(0,len(Age_)):
  if(Age_[x]=='ماع نم لقأ'):
    Age_[x]=-1

In [449]:
for x in range(0,len(Age_)):
  if(int(Age_[x]) > 100):
    print(Age_[x])
    print(Name_[x])
    Age_.pop(x)
    Serial_.pop(x)
    Gender_.pop(x)
    Identifier_.pop(x)
    Name_.pop(x)
#1823 is a valid data from the source however it is not logical, I will change it manually to 23

Instantiating the dataframe

In [467]:
df = pd.DataFrame({
    "Serial": Serial_,
    "Identification Number": Identifier_,
    "Name": Name_,
    "Gender": Gender_,
    "Age": Age_
})
convert_dict = {'Serial': int,
                'Identification Number': int,
                'Age': int,
                }

df = df.astype(convert_dict)

In [ ]:
count_male = len(df[df['Gender'] == 'ذكر'])
print(count_male)
count_female = len(df[df['Gender'] == 'انثى'])
print(count_female)
Mean_Age_Male = df[df['Gender'] == 'ذكر']['Age'].mean()
Mean_Age_Female = df[df['Gender'] == 'انثى']['Age'].mean()
print(Mean_Age_Female)
print(Mean_Age_Male)

In [ ]:
import matplotlib.pyplot as plt

grouped = df.groupby(['Gender']).size()

# Create a bar graph
grouped.plot(kind='bar', stacked=True)

# Set labels and title
plt.xlabel('Gender')
plt.ylabel('Number of Deaths')
plt.title('Deaths by Gender')

# Show the plot
plt.show()

In [ ]:
import plotly.express as px

fig = px.bar(df, x='Age', y='Gender',
             hover_data=['Gender', 'Age'], color='Gender',
             labels={'pop':'Deaths in gaza'}, height=400)
fig.show()

In [376]:
Infants = []
Children = []
Adolescents = []
Adults = []
Older_adults = []

Infants = df[(df['Age'] == -1) | (df['Age'] == 1)]
Children = df[(df['Age'] > 1) & (df['Age'] < 13)]
Adolescents = df[(df['Age'] > 12) & (df['Age'] < 18)]
Adults = df[(df['Age'] > 17) & (df['Age'] < 66)]
Older_adults = df[(df['Age'] > 64)]

In [ ]:
grouped = [len(Infants),len(Children),len(Adolescents),len(Adults),len(Older_adults)]
sizes = grouped
labels = ['Infants','Children','Adolescents','Adults','Older_adults']
plt.pie(sizes, labels=labels)
plt.title('Age groups of the martyrs')
plt.show()